# 140 - Experiment 1: Figures and metrics

In [1]:
import os, sys
from pathlib import Path

ENV_IS_GOOGLE_COLAB = True if 'google.colab' in str(get_ipython()) else False
os.environ["ENV_IS_GOOGLE_COLAB"] = str(ENV_IS_GOOGLE_COLAB)

if ENV_IS_GOOGLE_COLAB:
  from google.colab import drive
  mountpoint = Path("/content/drive")
  drive.mount(str(mountpoint)) # Mount gdrive to BASE
  base = mountpoint / "MyDrive/article_icdar_2023" # Adapt this to your situation
  sys.path.append(str(base)) # Add BASE to Python Path
  BASE = Path(base).resolve() # Make BASE absolute
  DATASETS =  BASE / "dataset"
else:
  BASE = Path(os.path.dirname(os.path.realpath("__file__"))).resolve() # If not on GColab, BASE will be the directory of this notebook
  DATASETS = Path('/work/stual/dataset_ICDAR').resolve()
  OUT_BASE = Path('/work/stual/res_ICDAR/method_1').resolve()

print(sys.path)
print(BASE)
print(DATASETS)
print(OUT_BASE)

['/lrde/home2/stual/stage_DAS/m1_independant_ner_layers', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages']
/lrde/home2/stual/stage_DAS/m1_independant_ner_layers
/work/stual/dataset_ICDAR
/work/stual/res_ICDAR/method_1


# Tools

In [2]:
import pandas as pd
import re
import os
import json

def compile_metrics(path): 
    rundirs = [f for f in os.listdir(path) if "run_" in f]

    df = pd.DataFrame()
    for run_dir in rundirs:
        run_path = path / run_dir
        nrun = re.search("\d+",run_dir)[0]
        
        files = [f for f in os.listdir(run_path) if "test_" in f and "level" not in f]
        sizes = [int(re.search("\d+",f)[0]) for f in files]
        
        for file, size in zip(files,sizes):
            file_path = run_path / file
            dftmp = pd.read_json(file_path, typ='series')
            dftmp = pd.DataFrame([dftmp])
            
            dftmp["trainsize"] = size 
            dftmp["run"] = nrun
            dftmp["trainsize_p"] = round(100 * size / 6084, 1)
            df = pd.concat([df, dftmp])

    return df.groupby(["run","trainsize"]).first()

# 141 - Experiment 1: figures and tables

In [3]:
from pathlib import Path

# PATHS
METRICS_DIR_E1 = OUT_BASE / "m1-110-experiment_1_metrics"
assert METRICS_DIR_E1
METRICS_DIR_E1

PosixPath('/work/stual/res_ICDAR/method_1/m1-110-experiment_1_metrics')

In [4]:
# Load Camembert IO metrics from metrics jsons
camembert_io_ref = compile_metrics(METRICS_DIR_E1 / "111-camembert-ner-multihead-io")
camembert_iob2_ref = compile_metrics(METRICS_DIR_E1 / "112-camembert-ner-multihead-iob2")
prtn_camembert_io_ref = compile_metrics(METRICS_DIR_E1 / "113-pretrained-camembert-ner-multihead-io")
prtn_camembert_iob2_ref = compile_metrics(METRICS_DIR_E1 / "114-pretrained-camembert-multihead-iob2")
metrics_raw_ref = pd.concat([camembert_io_ref,camembert_iob2_ref,prtn_camembert_io_ref,prtn_camembert_iob2_ref], keys=["CmBERT IO", "CmBERT IOB2", "Ptrn CmBERT IO", "Ptrn CmBERT IOB2"])
metrics_raw_ref

precision    recall        f1  accuracy  \
                 run trainsize                                            
CmBERT IO        1   6084        0.972824  0.965262  0.969028  0.973220   
                 2   6084        0.967221  0.960525  0.963861  0.962590   
                 3   6084        0.966561  0.961982  0.964266  0.968731   
                 4   6084        0.966630  0.957002  0.961792  0.965893   
                 5   6084        0.967119  0.961011  0.964055  0.967105   
CmBERT IOB2      1   6084        0.956222  0.960287  0.958250  0.961326   
                 2   6084        0.956647  0.964780  0.960697  0.964448   
                 3   6084        0.964122  0.969274  0.966691  0.969118   
                 4   6084        0.966943  0.962716  0.964825  0.964499   
                 5   6084        0.954426  0.966480  0.960415  0.967750   
Ptrn CmBERT IO   1   6084        0.971244  0.968177  0.969708  0.968473   
                 2   6084        0.959749  0.964412  0.962074  0.970382   
                 3   6084        0.965367  0.968298  0.966830  0.971543   
                 4   6084        0.962748  0.966841  0.964790  0.969634   
                 5   6084        0.969287  0.965991  0.967636  0.969350   
Ptrn CmBERT IOB2 1   6084        0.948804  0.963323  0.956008  0.960991   
                 2   6084        0.968674  0.965145  0.966906  0.970459   
                 3   6084        0.960308  0.969638  0.964950  0.970227   
                 4   6084        0.951371  0.964659  0.957969  0.963235   
                 5   6084        0.946853  0.962837  0.954778  0.967828   

                                precision-l1  recall-l1     f1-l1  \
                 run trainsize                                      
CmBERT IO        1   6084           0.964090   0.969397  0.966736   
                 2   6084           0.956407   0.961251  0.958823   
                 3   6084           0.959709   0.959709  0.959709   
                 4   6084           0.957480   0.956847  0.957163   
                 5   6084           0.955071   0.964113  0.959571   
CmBERT IOB2      1   6084           0.940630   0.959269  0.949858   
                 2   6084           0.947666   0.960810  0.954193   
                 3   6084           0.961572   0.969617  0.965578   
                 4   6084           0.955638   0.962792  0.959202   
                 5   6084           0.954368   0.966975  0.960630   
Ptrn CmBERT IO   1   6084           0.962713   0.972039  0.967353   
                 2   6084           0.943750   0.964113  0.953823   
                 3   6084           0.962825   0.969397  0.966100   
                 4   6084           0.963305   0.965214  0.964258   
                 5   6084           0.961480   0.967195  0.964329   
Ptrn CmBERT IOB2 1   6084           0.941202   0.958609  0.949825   
                 2   6084           0.959834   0.968076  0.963937   
                 3   6084           0.957879   0.966314  0.962078   
                 4   6084           0.950970   0.960810  0.955865   
                 5   6084           0.930769   0.959049  0.944697   

                                accuracy-l1  precision-l2  recall-l2  ...  \
                 run trainsize                                        ...   
CmBERT IO        1   6084          0.970846      0.983898   0.960173  ...   
                 2   6084          0.951032      0.980892   0.959632  ...   
                 3   6084          0.962590      0.975082   0.964779  ...   
                 4   6084          0.957224      0.978128   0.957193  ...   
                 5   6084          0.961094      0.982481   0.957193  ...   
CmBERT IOB2      1   6084          0.948297      0.976079   0.961538  ...   
                 2   6084          0.951548      0.967829   0.969664  ...   
                 3   6084          0.961507      0.967280   0.968852  ...   
                 4   6084          0.955573      0.981226   0.962622  ...   
               

In [5]:
# Store p/r/f1 as percentages
eval_ = ["precision-l1","recall-l1","f1-l1","precision-l2","recall-l2","f1-l2","precision","recall","f1","precision-jl","recall_jl","f1_jl"]
metrics_ref = metrics_raw_ref.copy()
metrics_ref[eval_] = metrics_raw_ref[eval_].multiply(100., axis=1)
metrics_ref.head()

precision     recall         f1  accuracy  \
          run trainsize                                              
CmBERT IO 1   6084       97.282409  96.526175  96.902817  0.973220   
          2   6084       96.722114  96.052472  96.386130  0.962590   
          3   6084       96.656090  96.198227  96.426615  0.968731   
          4   6084       96.662986  95.700231  96.179199  0.965893   
          5   6084       96.711893  96.101057  96.405507  0.967105   

                         precision-l1  recall-l1      f1-l1  accuracy-l1  \
          run trainsize                                                    
CmBERT IO 1   6084          96.409021  96.939674  96.673619     0.970846   
          2   6084          95.640745  96.125055  95.882288     0.951032   
          3   6084          95.970938  95.970938  95.970938     0.962590   
          4   6084          95.747962  95.684720  95.716331     0.957224   
          5   6084          95.507088  96.411273  95.957051     0.961094   

                         precision-l2  recall-l2  ...  \
          run trainsize                           ...   
CmBERT IO 1   6084          98.389783  96.017339  ...   
          2   6084          98.089172  95.963154  ...   
          3   6084          97.508215  96.477919  ...   
          4   6084          97.812846  95.719317  ...   
          5   6084          98.248053  95.719317  ...   

                                                                    ACT-l2  \
          run trainsize                                                      
CmBERT IO 1   6084       {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, '...   
          2   6084       {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, '...   
          3   6084       {'precision': 0.2894736842105263, 'recall': 0....   
          4   6084       {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, '...   
          5   6084       {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, '...   

                                                                      DESC  \
          run trainsize                                                      
CmBERT IO 1   6084       {'precision': 0.5319148936170213, 'recall': 0....   
          2   6084       {'precision': 0.3504273504273504, 'recall': 0....   
          3   6084       {'precision': 0.5188679245283019, 'recall': 0....   
          4   6084       {'precision': 0.39849624060150374, 'recall': 0...   
          5   6084       {'precision': 0.39473684210526316, 'recall': 0...   

                                                                    TITREH  \
          run trainsize                                                      
CmBERT IO 1   6084       {'precision': 0.9772727272727273, 'recall': 1....   
          2   6084       {'precision': 0.9761904761904762, 'recall': 0....   
          3   6084       {'precision': 0.9772727272727273, 'recall': 1....   
          4   6084       {'precision': 0.9772727272727273, 'recall': 1....   
          5   6084       {'precision': 0.9767441860465116, 'recall': 0....   

                                                                    TITREP  \
          run trainsize                                                      
CmBERT IO 1   6084       {'precision': 0.4074074074074074, 'recall': 0....   
          2   6084       {'precision': 0.7647058823529411, 'recall': 0....   
          3   6084       {'precision': 0.6666666666666666, 'recall': 0....   
          4   6084       {'precision': 0.4, 'recall': 0.303030303030303...   
          5   6084       {'precision': 1.0, 'recall': 0.030303030303030...   

                                                                      SPAT  \
          run trainsize                                                      
CmBERT IO 1   6084       {'precision': 0.9805491990846682, 'recall': 0....   
          2   6084       {'precision': 0.9732954545454545, 'recall': 0....   
          3   6084       {'precision': 0.9771428571428571, 'recall': 0....   
          4   6084     

### Build the average table

In [20]:
# Average over runs
averaged_ref = metrics_ref.groupby(level=0).apply(lambda grp: grp.groupby(by="trainsize").mean())
averaged_ref.set_index(["trainsize_p"], append=True,inplace=True)

# Keep just the necessary columns
averaged_ref=averaged_ref[["f1-l1","f1-l2","f1","f1_jl"]]

# Set pretty names
averaged_ref.index.names = ['Model','Trainset Size',"%"]
averaged_ref.rename(columns={"f1-l1":"Level 1",
    "f1-l2":"Level 2",
    "f1":"Global",
    "f1_jl":"P+L1+P+L2"
                         }, errors="raise", inplace=True)
averaged_ref.rename(mapper={"camembert_io_ref": "CmBERT IO","camembert_iob2_ref": "CmBERT IOB2","prtn_camembert_io_ref": "Ptrn CmBERT IO","prtn_camembert_iob2_ref": "Ptrn CmBERT IOB2"}, errors="ignore", inplace=True, axis=0)
averaged_ref

,,,Level 1,Level 2,Global,P+L1+P+L2
Model,Trainset Size,%,,,,
CmBERT IO,6084,100.0,96.040045,96.983179,96.460054,95.699654
CmBERT IOB2,6084,100.0,95.789203,96.751012,96.217547,95.663863
Ptrn CmBERT IO,6084,100.0,96.317256,96.999829,96.620776,95.918409
Ptrn CmBERT IOB2,6084,100.0,95.528058,96.612880,96.012234,95.331939


### Create the results table

In [21]:
# Filter and transpose to obtain the latex table
latex_table_ref = averaged_ref.stack().unstack(level=[1,2])

# Swap model name and metrics to get a nice table
latex_table_ref = latex_table_ref.swaplevel(0,1).sort_index(level=0)

caption = "F1 score measured on the fine-tuned models CmBERT, CmBERT+ptrn on reference dataset with Independent Flat NER layers approach (M1)."
print(latex_table_ref.to_latex(float_format="%.1f", multirow=True, caption=caption))
latex_table_ref

\begin{table}
\centering
\caption{F1 score measured on the fine-tuned models CmBERT, CmBERT+ptrn on reference dataset.}
\begin{tabular}{llr}
\toprule
          & Trainset Size &  6084 \\
          & \% & 100.0 \\
{} & Model &       \\
\midrule
\multirow{4}{*}{Global} & CmBERT IO &  96.5 \\
          & CmBERT IOB2 &  96.2 \\
          & Ptrn CmBERT IO &  96.6 \\
          & Ptrn CmBERT IOB2 &  96.0 \\
\cline{1-3}
\multirow{4}{*}{Level 1} & CmBERT IO &  96.0 \\
          & CmBERT IOB2 &  95.8 \\
          & Ptrn CmBERT IO &  96.3 \\
          & Ptrn CmBERT IOB2 &  95.5 \\
\cline{1-3}
\multirow{4}{*}{Level 2} & CmBERT IO &  97.0 \\
          & CmBERT IOB2 &  96.8 \\
          & Ptrn CmBERT IO &  97.0 \\
          & Ptrn CmBERT IOB2 &  96.6 \\
\cline{1-3}
\multirow{4}{*}{P+L1+P+L2} & CmBERT IO &  95.7 \\
          & CmBERT IOB2 &  95.7 \\
          & Ptrn CmBERT IO &  95.9 \\
          & Ptrn CmBERT IOB2 &  95.3 \\
\bottomrule
\end{tabular}
\end{table}



Trainset Size                    6084
%                               100.0
          Model                      
Global    CmBERT IO         96.460054
          CmBERT IOB2       96.217547
          Ptrn CmBERT IO    96.620776
          Ptrn CmBERT IOB2  96.012234
Level 1   CmBERT IO         96.040045
          CmBERT IOB2       95.789203
          Ptrn CmBERT IO    96.317256
          Ptrn CmBERT IOB2  95.528058
Level 2   CmBERT IO         96.983179
          CmBERT IOB2       96.751012
          Ptrn CmBERT IO    96.999829
          Ptrn CmBERT IOB2  96.612880
P+L1+P+L2 CmBERT IO         95.699654
          CmBERT IOB2       95.663863
          Ptrn CmBERT IO    95.918409
          Ptrn CmBERT IOB2  95.331939

# 142 - Experiment 2: figures and tables

In [8]:
from pathlib import Path

# PATHS
METRICS_DIR_E2 = OUT_BASE / "m1-120-experiment_2_metrics"
assert METRICS_DIR_E2
METRICS_DIR_E2

PosixPath('/work/stual/res_ICDAR/method_1/m1-120-experiment_2_metrics')

In [9]:
# Load Camembert IO metrics from metrics jsons
camembert_io_pero = compile_metrics(METRICS_DIR_E2 / "121-camembert-ner-multihead-io")
camembert_iob2_pero = compile_metrics(METRICS_DIR_E2 / "122-camembert-ner-multihead-iob2")
prtn_camembert_io_pero = compile_metrics(METRICS_DIR_E2 / "123-pretrained-camembert-ner-multihead-io")
prtn_camembert_iob2_pero = compile_metrics(METRICS_DIR_E2 / "124-pretrained-camembert-multihead-iob2")
metrics_raw_pero = pd.concat([camembert_io_pero,camembert_iob2_pero,prtn_camembert_io_pero,prtn_camembert_iob2_pero], keys=["CmBERT IO", "CmBERT IOB2", "Ptrn CmBERT IO", "Ptrn CmBERT IOB2"])
metrics_raw_pero

precision    recall        f1  accuracy  \
                 run trainsize                                            
CmBERT IO        1   6084        0.938887  0.940598  0.939741  0.960667   
                 2   6084        0.934635  0.936224  0.935429  0.958684   
                 3   6084        0.932882  0.937075  0.934974  0.958605   
                 4   6084        0.937803  0.939626  0.938714  0.960067   
                 5   6084        0.939239  0.938897  0.939068  0.957509   
CmBERT IOB2      1   6084        0.926027  0.944134  0.934993  0.953698   
                 2   6084        0.937235  0.939398  0.938315  0.955551   
                 3   6084        0.929223  0.940734  0.934943  0.957613   
                 4   6084        0.931254  0.942677  0.936930  0.954220   
                 5   6084        0.926479  0.933568  0.930010  0.953672   
Ptrn CmBERT IO   1   6084        0.939128  0.950194  0.944629  0.961920   
                 2   6084        0.942025  0.943513  0.942769  0.961398   
                 3   6084        0.945634  0.944485  0.945059  0.961137   
                 4   6084        0.937590  0.948980  0.943250  0.962442   
                 5   6084        0.933645  0.946914  0.940233  0.958892   
Ptrn CmBERT IOB2 1   6084        0.940317  0.945227  0.942765  0.954821   
                 2   6084        0.939405  0.947049  0.943211  0.960954   
                 3   6084        0.929192  0.948263  0.938631  0.953646   
                 4   6084        0.940152  0.944377  0.942260  0.957039   
                 5   6084        0.929441  0.945470  0.937387  0.956622   

                                precision-l1  recall-l1     f1-l1  \
                 run trainsize                                      
CmBERT IO        1   6084           0.927178   0.941876  0.934469   
                 2   6084           0.918668   0.935051  0.926787   
                 3   6084           0.920779   0.936592  0.928618   
                 4   6084           0.924446   0.937472  0.930914   
                 5   6084           0.927998   0.942096  0.934994   
CmBERT IOB2      1   6084           0.927190   0.939234  0.933173   
                 2   6084           0.928772   0.944518  0.936579   
                 3   6084           0.921314   0.938353  0.929756   
                 4   6084           0.920215   0.942096  0.931027   
                 5   6084           0.918022   0.931968  0.924943   
Ptrn CmBERT IO   1   6084           0.939420   0.949141  0.944256   
                 2   6084           0.933391   0.944077  0.938704   
                 3   6084           0.935344   0.949141  0.942192   
                 4   6084           0.936156   0.949141  0.942604   
                 5   6084           0.931715   0.946279  0.938940   
Ptrn CmBERT IOB2 1   6084           0.929757   0.950022  0.939780   
                 2   6084           0.934882   0.948261  0.941524   
                 3   6084           0.928479   0.948921  0.938589   
                 4   6084           0.930102   0.946279  0.938121   
                 5   6084           0.917382   0.941215  0.929146   

                                accuracy-l1  precision-l2  recall-l2  ...  \
                 run trainsize                                        ...   
CmBERT IO        1   6084          0.955108      0.953757   0.939024  ...   
                 2   6084          0.951663      0.955010   0.937669  ...   
                 3   6084          0.949157      0.948205   0.937669  ...   
                 4   6084          0.951558      0.954695   0.942276  ...   
                 5   6084          0.951976      0.953566   0.934959  ...   
CmBERT IOB2      1   6084          0.945138      0.924618   0.950163  ...   
                 2   6084          0.950410      0.947991   0.933099  ...   
                 3   6084          0.950880      0.939084   0.943662  ...   
                 4   6084          0.947382      0.945183   0.943391  ...   
               

In [10]:
# Store p/r/f1 as percentages
eval_ = ["precision-l1","recall-l1","f1-l1","precision-l2","recall-l2","f1-l2","precision","recall","f1","precision-jl","recall_jl","f1_jl"]
metrics_pero = metrics_raw_pero.copy()
metrics_pero[eval_] = metrics_raw_pero[eval_].multiply(100., axis=1)
metrics_pero.head()

precision     recall         f1  accuracy  \
          run trainsize                                              
CmBERT IO 1   6084       93.888687  94.059767  93.974149  0.960667   
          2   6084       93.463497  93.622449  93.542906  0.958684   
          3   6084       93.288185  93.707483  93.497364  0.958605   
          4   6084       93.780310  93.962585  93.871359  0.960067   
          5   6084       93.923928  93.889699  93.906810  0.957509   

                         precision-l1  recall-l1      f1-l1  accuracy-l1  \
          run trainsize                                                    
CmBERT IO 1   6084          92.717815  94.187583  93.446920     0.955108   
          2   6084          91.866753  93.505064  92.678669     0.951663   
          3   6084          92.077922  93.659181  92.861821     0.949157   
          4   6084          92.444637  93.747248  93.091386     0.951558   
          5   6084          92.799827  94.209599  93.499399     0.951976   

                         precision-l2  recall-l2  ...  \
          run trainsize                           ...   
CmBERT IO 1   6084          95.375723  93.902439  ...   
          2   6084          95.500966  93.766938  ...   
          3   6084          94.820499  93.766938  ...   
          4   6084          95.469522  94.227642  ...   
          5   6084          95.356551  93.495935  ...   

                                                                    ACT-l2  \
          run trainsize                                                      
CmBERT IO 1   6084       {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, '...   
          2   6084       {'precision': 0.6428571428571429, 'recall': 0....   
          3   6084       {'precision': 0.3333333333333333, 'recall': 0....   
          4   6084       {'precision': 0.48717948717948717, 'recall': 0...   
          5   6084       {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, '...   

                                                                      DESC  \
          run trainsize                                                      
CmBERT IO 1   6084       {'precision': 0.4727272727272727, 'recall': 0....   
          2   6084       {'precision': 0.4936708860759494, 'recall': 0....   
          3   6084       {'precision': 0.3739130434782609, 'recall': 0....   
          4   6084       {'precision': 0.4142857142857143, 'recall': 0....   
          5   6084       {'precision': 0.5568181818181818, 'recall': 0....   

                                                                    TITREH  \
          run trainsize                                                      
CmBERT IO 1   6084       {'precision': 0.6904761904761905, 'recall': 0....   
          2   6084       {'precision': 0.8285714285714286, 'recall': 0....   
          3   6084       {'precision': 0.6222222222222222, 'recall': 0....   
          4   6084       {'precision': 0.7368421052631579, 'recall': 0....   
          5   6084       {'precision': 0.75, 'recall': 0.69767441860465...   

                                                                    TITREP  \
          run trainsize                                                      
CmBERT IO 1   6084       {'precision': 0.5333333333333333, 'recall': 0....   
          2   6084       {'precision': 0.6521739130434783, 'recall': 0....   
          3   6084       {'precision': 0.65, 'recall': 0.40625, 'f1': 0...   
          4   6084       {'precision': 0.631578947368421, 'recall': 0.3...   
          5   6084       {'precision': 0.5625, 'recall': 0.28125, 'f1':...   

                                                                      SPAT  \
          run trainsize                                                      
CmBERT IO 1   6084       {'precision': 0.9438138479001135, 'recall': 0....   
          2   6084       {'precision': 0.9388101983002833, 'recall': 0....   
          3   6084       {'precision': 0.9404424276800908, 'recall': 0....   
          4   6084     

### Build the averaged table


In [11]:
# Average over runs
averaged_pero = metrics_pero.groupby(level=0).apply(lambda grp: grp.groupby(by="trainsize").mean())
averaged_pero.set_index(["trainsize_p"], append=True,inplace=True)

# Keep just the necessary columns
averaged_pero=averaged_pero[["f1-l1","f1-l2","f1","f1_jl"]]

# Set pretty names
averaged_pero.index.names = ['Model','Trainset Size',"%"]
averaged_pero.rename(columns={
    "f1-l1":"Level 1",
    "f1-l2":"Level 2",
    "f1":"Global",
    "f1_jl":"P+L1+P+L2"}, errors="raise", inplace=True)
averaged_pero.rename(mapper={"camembert_io_ref": "CmBERT IO","camembert_iob2_ref": "CmBERT IOB2","prtn_camembert_io_ref": "Ptrn CmBERT IO","prtn_camembert_iob2_ref": "Ptrn CmBERT IOB2"}, errors="ignore", inplace=True, axis=0)
averaged_pero

,,,Level 1,Level 2,Global,P+L1+P+L2
Model,Trainset Size,%,,,,
CmBERT IO,6084,100.0,93.115639,94.562345,93.758518,93.355103
CmBERT IOB2,6084,100.0,93.109544,93.993053,93.503830,93.127077
Ptrn CmBERT IO,6084,100.0,94.133931,94.548789,94.318797,93.835619
Ptrn CmBERT IOB2,6084,100.0,93.743182,94.510817,94.085092,93.708233


### Create the results table

In [12]:
# Filter and transpose to obtain the latex table
latex_table_pero = averaged_pero.stack().unstack(level=[1,2])

# Swap model name and metrics to get a nice table
latex_table_pero = latex_table_pero.swaplevel(0,1).sort_index(level=0)

caption = "F1 score measured on the fine-tuned models CmBERT, CmBERT+ptrn on noisy dataset with Independent Flat NER layers approach (M1)."
print(latex_table_pero.to_latex(float_format="%.1f", multirow=True, caption=caption))

\begin{table}
\centering
\caption{F1 score measured on the fine-tuned models CmBERT, CmBERT+ptrn on noisy dataset.}
\begin{tabular}{llr}
\toprule
          & Trainset Size &  6084 \\
          & \% & 100.0 \\
{} & Model &       \\
\midrule
\multirow{4}{*}{Global} & CmBERT IO &  93.8 \\
          & CmBERT IOB2 &  93.5 \\
          & Ptrn CmBERT IO &  94.3 \\
          & Ptrn CmBERT IOB2 &  94.1 \\
\cline{1-3}
\multirow{4}{*}{Level 1} & CmBERT IO &  93.1 \\
          & CmBERT IOB2 &  93.1 \\
          & Ptrn CmBERT IO &  94.1 \\
          & Ptrn CmBERT IOB2 &  93.7 \\
\cline{1-3}
\multirow{4}{*}{Level 2} & CmBERT IO &  94.6 \\
          & CmBERT IOB2 &  94.0 \\
          & Ptrn CmBERT IO &  94.5 \\
          & Ptrn CmBERT IOB2 &  94.5 \\
\cline{1-3}
\multirow{4}{*}{P+L1+P+L2} & CmBERT IO &  93.4 \\
          & CmBERT IOB2 &  93.1 \\
          & Ptrn CmBERT IO &  93.8 \\
          & Ptrn CmBERT IOB2 &  93.7 \\
\bottomrule
\end{tabular}
\end{table}



# 143 - Experiments 1 and 2 results table

In [43]:
averaged = pd.concat([averaged_ref,averaged_pero],keys=["Reference","Pero OCR"])
averaged = averaged.reset_index(level=[2,3], drop=True)
averaged

Level 1    Level 2     Global  P+L1+P+L2
          Model                                                       
Reference CmBERT IO         96.040045  96.983179  96.460054  95.699654
          CmBERT IOB2       95.789203  96.751012  96.217547  95.663863
          Ptrn CmBERT IO    96.317256  96.999829  96.620776  95.918409
          Ptrn CmBERT IOB2  95.528058  96.612880  96.012234  95.331939
Pero OCR  CmBERT IO         93.115639  94.562345  93.758518  93.355103
          CmBERT IOB2       93.109544  93.993053  93.503830  93.127077
          Ptrn CmBERT IO    94.133931  94.548789  94.318797  93.835619
          Ptrn CmBERT IOB2  93.743182  94.510817  94.085092  93.708233

In [45]:
caption = "F1 score measured on the fine-tuned models CmBERTand CmBERT+ptrn on reference dataset and noisy dataset with Independent Flat NER layers approach (M1)."
print(averaged.to_latex(float_format="%.1f", multirow=True, caption=caption))
averaged

\begin{table}
\centering
\caption{F1 score measured on the fine-tuned models CmBERTand CmBERT+ptrn on reference dataset and noisy dataset with method 1.}
\begin{tabular}{llrrrr}
\toprule
         &                  &  Level 1 &  Level 2 &  Global &  P+L1+P+L2 \\
{} & Model &          &          &         &            \\
\midrule
\multirow{4}{*}{Reference} & CmBERT IO &     96.0 &     97.0 &    96.5 &       95.7 \\
         & CmBERT IOB2 &     95.8 &     96.8 &    96.2 &       95.7 \\
         & Ptrn CmBERT IO &     96.3 &     97.0 &    96.6 &       95.9 \\
         & Ptrn CmBERT IOB2 &     95.5 &     96.6 &    96.0 &       95.3 \\
\cline{1-6}
\multirow{4}{*}{Pero OCR} & CmBERT IO &     93.1 &     94.6 &    93.8 &       93.4 \\
         & CmBERT IOB2 &     93.1 &     94.0 &    93.5 &       93.1 \\
         & Ptrn CmBERT IO &     94.1 &     94.5 &    94.3 &       93.8 \\
         & Ptrn CmBERT IOB2 &     93.7 &     94.5 &    94.1 &       93.7 \\
\bottomrule
\end{tabular}
\end{table}



Level 1    Level 2     Global  P+L1+P+L2
          Model                                                       
Reference CmBERT IO         96.040045  96.983179  96.460054  95.699654
          CmBERT IOB2       95.789203  96.751012  96.217547  95.663863
          Ptrn CmBERT IO    96.317256  96.999829  96.620776  95.918409
          Ptrn CmBERT IOB2  95.528058  96.612880  96.012234  95.331939
Pero OCR  CmBERT IO         93.115639  94.562345  93.758518  93.355103
          CmBERT IOB2       93.109544  93.993053  93.503830  93.127077
          Ptrn CmBERT IO    94.133931  94.548789  94.318797  93.835619
          Ptrn CmBERT IOB2  93.743182  94.510817  94.085092  93.708233